## Documentation

To read more about the bulk API, visit the [docs](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-bulk.html).

![bulk_api_docs](../images/bulk_api_docs.png)

## Connect to ElasticSearch


In [1]:
import { Client } from "npm:@elastic/elasticsearch";
import { load } from "https://deno.land/std/dotenv/mod.ts";

const env = await load({ envPath: "../.env" });

const client = new Client({
  node: env.ELASTICSEARCH_NODE,
  auth: {
    apiKey: env.ELASTICSEARCH_API_KEY,
  },
});

const info = await client.info();
console.log(info);


{
  name: "1e6d9c093984",
  cluster_name: "docker-cluster",
  cluster_uuid: "v3fUyW9OReext6IjPiOCqg",
  version: {
    number: "8.17.4",
    build_flavor: "default",
    build_type: "docker",
    build_hash: "c63c7f5f8ce7d2e4805b7b3d842e7e792d84dda1",
    build_date: "2025-03-20T15:39:59.811110136Z",
    build_snapshot: false,
    lucene_version: "9.12.0",
    minimum_wire_compatibility_version: "7.17.0",
    minimum_index_compatibility_version: "7.0.0"
  },
  tagline: "You Know, for Search"
}


## Create index


In [7]:
await client.indices.delete({ index: "my_index", ignore_unavailable: true });
await client.indices.create({ index: "my_index" });


{ acknowledged: true, shards_acknowledged: true, index: "my_index" }

Without bulk API we executed each operation one at a time, with each action requiring a separate API call. This approach is slow and inefficient. Now, let’s see how to accomplish the same task using the bulk API.

## Bulk API

Provides a way to perform multiple `index`, `create`, `delete`, and `update` actions in a single request.

The actions are specified in the request body using a newline delimited JSON (NDJSON) structure:
\
\
`action_and_meta_data\n`
\
`optional_source\n`
\
`action_and_meta_data\n`
\
`optional_source\n`
\
`....`
\
`action_and_meta_data\n`
\
`optional_source\n`

In [17]:
const operations = [
  // Action 1
  {
    index: {
      _index: "my_index",
      _id: "1",
    },
  },
  // Source 1
  {
    title: "Title 1",
    text: "This is the first sample document text.",
    createdAt: "2025-03-01",
  },
  // Action 2
  {
    index: {
      _index: "my_index",
      _id: "2",
    },
  },
  // Source 2
  {
    title: "Title 2",
    text: "Here is another example of a document.",
    createdAt: "2025-03-02",
  },
  // Action 3
  {
    index: {
      _index: "my_index",
      _id: "3",
    },
  },
  // Source 3
  {
    title: "Title 3",
    text: "The content of the third document goes here.",
    createdAt: "2025-03-03",
  },
  // Action 4
  {
    update: {
      _id: "1",
      _index: "my_index",
    },
  },
  // Source 4
  {
    doc: {
      title: "New Title 1",
    },
  },
  // Action 5
  {
    update: {
      _id: "2",
      _index: "my_index",
    },
  },
  // Source 5
  {
    doc: {
      new_field: "New Field",
    },
  },
  // Action 6
  {
    delete: {
      _index: "my_index",
      _id: "3",
    },
  },
];

const response = await client.bulk({
  operations,
});

console.log(response);


{
  errors: false,
  took: 0,
  items: [
    {
      index: {
        _index: "my_index",
        _id: "1",
        _version: 7,
        result: "updated",
        _shards: { total: 2, successful: 1, failed: 0 },
        _seq_no: 18,
        _primary_term: 1,
        status: 200
      }
    },
    {
      index: {
        _index: "my_index",
        _id: "2",
        _version: 7,
        result: "updated",
        _shards: { total: 2, successful: 1, failed: 0 },
        _seq_no: 19,
        _primary_term: 1,
        status: 200
      }
    },
    {
      index: {
        _index: "my_index",
        _id: "3",
        _version: 3,
        result: "created",
        _shards: { total: 2, successful: 1, failed: 0 },
        _seq_no: 20,
        _primary_term: 1,
        status: 201
      }
    },
    {
      update: {
        _index: "my_index",
        _id: "1",
        _version: 8,
        result: "updated",
        _shards: { total: 2, successful: 1, failed: 0 },
        _seq_no: 21,
   

If `errors` is `false`, it means the bulk API successfully executed all the actions.

In [15]:
response.errors

false